#Step1
Import libraries

*Note:* Please, make sure that the next pacakages are available:
*numpy
*pandas
*matplotlib
*requests
*sklearn
*beautifulsoup4

In [1]:
import pandas as pd # library to process data as dataframes
import numpy as np
import matplotlib.pyplot as plt # plotting library
import requests
# backend for rendering plots within the browser
%matplotlib inline 
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


The next lines of code read the Wikipedia post with the list of post codes of Canada and creates a Pandas data frame with them


In [2]:
website_cotent_in_html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text


soup = BeautifulSoup(website_cotent_in_html,"html.parser")
my_table = soup.find('table',{'class':'wikitable sortable'})

table_rows=my_table.find_all('tr')

table_data = []

for row in table_rows:
    table_data.append([t.text.strip() for t in row.find_all('td')])

postal_codes_raw_df = pd.DataFrame(table_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
#df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
print(postal_codes_raw_df.shape)
postal_codes_raw_df.head(10)

(181, 3)


,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


Now is time to clean the data and prepare it in the desired shape:


In [3]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
postal_codes_df =  postal_codes_raw_df[postal_codes_raw_df['Borough'] != "Not assigned"]
postal_codes_df.replace('None', np.nan, inplace = True)
postal_codes_df.dropna(inplace=True)
print(postal_codes_df.shape)
postal_codes_df.head(5)

(103, 3)


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

postal_codes_raw_df.groupby('PostalCode').count().sort_values(by="Borough", ascending=False)

,Borough,Neighbourhood
PostalCode,,
M1A,1,1
M6S,1,1
M6V,1,1
M6W,1,1
M6X,1,1
...,...,...
M4B,1,1
M4C,1,1
M4E,1,1


In [5]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
